here i attempt to write a REST2 factory for a canonical openmm.system (i.e. containing `HarmonicBondForce`, `HarmonicAngleForce`, `PeriodicTorsionForce`, `NonbondedForce`)

In [1]:
from openmmtools.testsystems import AlanineDipeptideVacuum

define a vacuum alanine dipeptide

In [2]:
ala = AlanineDipeptideVacuum()
sys = ala.system

In [3]:
from perses.annihilation.rest import RESTTopologyFactory

query the topology

In [4]:
ala.topology
for atom in ala.topology.atoms():
    print(atom)

<Atom 0 (H1) of chain 0 residue 0 (ACE)>
<Atom 1 (CH3) of chain 0 residue 0 (ACE)>
<Atom 2 (H2) of chain 0 residue 0 (ACE)>
<Atom 3 (H3) of chain 0 residue 0 (ACE)>
<Atom 4 (C) of chain 0 residue 0 (ACE)>
<Atom 5 (O) of chain 0 residue 0 (ACE)>
<Atom 6 (N) of chain 0 residue 1 (ALA)>
<Atom 7 (H) of chain 0 residue 1 (ALA)>
<Atom 8 (CA) of chain 0 residue 1 (ALA)>
<Atom 9 (HA) of chain 0 residue 1 (ALA)>
<Atom 10 (CB) of chain 0 residue 1 (ALA)>
<Atom 11 (HB1) of chain 0 residue 1 (ALA)>
<Atom 12 (HB2) of chain 0 residue 1 (ALA)>
<Atom 13 (HB3) of chain 0 residue 1 (ALA)>
<Atom 14 (C) of chain 0 residue 1 (ALA)>
<Atom 15 (O) of chain 0 residue 1 (ALA)>
<Atom 16 (N) of chain 0 residue 2 (NME)>
<Atom 17 (H) of chain 0 residue 2 (NME)>
<Atom 18 (C) of chain 0 residue 2 (NME)>
<Atom 19 (H1) of chain 0 residue 2 (NME)>
<Atom 20 (H2) of chain 0 residue 2 (NME)>
<Atom 21 (H3) of chain 0 residue 2 (NME)>


I'll make the ALA residue 'solute' for REST2

remove the `CMMForce`

In [5]:
ala.system.removeForce(4)

make the factory

In [6]:
factory = RESTTopologyFactory(ala.system, solute_region=list(range(6, 16)))

INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:REST:No unknown forces.


pull the system

In [7]:
REST_system = factory.REST_system

query the forces to make sure it _looks_ right

In [8]:
REST_system.getForces()

[<simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b7866bf2b10> >,
 <simtk.openmm.openmm.CustomAngleForce; proxy of <Swig Object of type 'OpenMM::CustomAngleForce *' at 0x2b7866bf2930> >,
 <simtk.openmm.openmm.CustomTorsionForce; proxy of <Swig Object of type 'OpenMM::CustomTorsionForce *' at 0x2b789bd71390> >,
 <simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2b789bd71300> >,
 <simtk.openmm.openmm.CustomNonbondedForce; proxy of <Swig Object of type 'OpenMM::CustomNonbondedForce *' at 0x2b789bd88cf0> >,
 <simtk.openmm.openmm.CustomBondForce; proxy of <Swig Object of type 'OpenMM::CustomBondForce *' at 0x2b789bd88d20> >]

all of the nonbonded forces must have the same number of particles

In [9]:
num_particles = REST_system.getNumParticles()
assert num_particles == factory._out_system_forces['CustomNonbondedForce'].getNumParticles()
assert num_particles == factory._out_system_forces['NonbondedForce'].getNumParticles()

## TODO:

here is a few things that have to be done...

- Given the default Global variables, do energy component bookkeeping with the REST system and the original system. Specifically, the first `CustomBondForce` must match the `HarmonicBondForce` (same with the second and third forces with the `HarmonicAngle` and `PeriodicTorsionForce`s). The sum of the `NonbondedForce`, the `CustomNonbondedForce`, and the `CustomBondForce` (last one) must be equal to the energy of the original system's `NonbondedForce`.

- Create a subclass of the `AlchemicalState` to implement the appropriate parameters for `solute_scale` and `inter_scale` (which are `GlobalParameters` of the `REST_system`) and perform Repex.

- Run REST2!

## Run energy component bookkeeping

In [10]:
from openmmtools.states import CompoundThermodynamicState, SamplerState, ThermodynamicState
from perses.annihilation.lambda_protocol import RelativeAlchemicalState, LambdaProtocol
from openmmtools.integrators import LangevinIntegrator
from simtk import unit
from openmmtools import cache
temp = 300 * unit.kelvin
from perses.tests.utils import compute_potential_components
import copy
from openmmtools.constants import kB
beta = 1/(kB * temp)
from perses.dispersed.feptasks import minimize
from coddiwomple.openmm.states import OpenMMPDFState
from perses.dispersed.utils import configure_platform
from coddiwomple.openmm.utils import get_dummy_integrator
from openmmtools import utils

conducting subsequent work with the following platform: CUDA


In [11]:
from perses.tests.test_topology_proposal import generate_atp

INFO:rdkit:Enabling RDKit 2020.09.1 jupyter extensions


Potential components for the vanilla ala system

In [12]:
thermostate = ThermodynamicState(system=ala.system, temperature=temp)
sampler_state = SamplerState(ala.positions, box_vectors = ala.system.getDefaultPeriodicBoxVectors())
integrator = get_dummy_integrator()
platform = configure_platform(utils.get_fastest_platform().getName())
context = thermostate.create_context(integrator, platform=platform)
minimize(thermostate, sampler_state)
thermostate.reduced_potential(sampler_state)
sampler_state.apply_to_context(context)
print(compute_potential_components(context, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('HarmonicBondForce', 0.5874003195280203), ('HarmonicAngleForce', 1.209485556878217), ('PeriodicTorsionForce', 2.321496276111238), ('NonbondedForce', -50.94914523261006), ('AndersenThermostat', 0.0)]


Potential components for the REST system

In [13]:
thermostate = ThermodynamicState(system=REST_system, temperature=temp)
# Don't re-initialize the sampler state 
integrator = get_dummy_integrator()
platform = configure_platform(utils.get_fastest_platform().getName())
context = thermostate.create_context(integrator, platform=platform)
# Don't minimize again
thermostate.reduced_potential(sampler_state)
sampler_state.apply_to_context(context)
print(compute_potential_components(context, beta=beta))

conducting subsequent work with the following platform: CUDA
conducting subsequent work with the following platform: CUDA
[('CustomBondForce', 0.5874003195280203), ('CustomAngleForce', 1.2094855568782168), ('CustomTorsionForce', 2.3214942161284062), ('NonbondedForce', -1.187543685095446), ('CustomNonbondedForce', -133.23778625838835), ('CustomBondForce', 83.4761858046491), ('AndersenThermostat', 0.0)]


In [ ]:
# REST Nonbonded + CustomNonbonded + CustomBond = -50.949

## Run REST2 on capped THR->ALA (flattened torsions/exceptions) in vacuum

In [13]:
import pickle
import os
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import openmm, unit
import math
from openmmtools.constants import kB
from openmmtools import mcmc, multistate
import copy

In [2]:
# Load in r-htf at endstate = 0
out_dir = "/data/chodera/zhangi/perses_benchmark/neq/11/0"
htf = pickle.load(open(os.path.join(out_dir, f"0_vacuum_0.pickle"), "rb" ))


DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=None, lineno=6)
           2	LOAD_GLOBAL(arg=0, lineno=6)
           4	LOAD_METHOD(arg=1, lineno=6)
           6	LOAD_CONST(arg=1, lineno=6)
           8	CALL_METHOD(arg=1, lineno=6)
          10	STORE_FAST(arg=2, lineno=6)
          12	LOAD_FAST(arg=0, lineno=7)
          14	LOAD_CONST(arg=2, lineno=7)
          16	BINARY_SUBSCR(arg=None, lineno=7)
          18	LOAD_FAST(arg=1, lineno=7)
          20	LOAD_CONST(arg=3, lineno=7)
          22	BINARY_SUBSCR(arg=None, lineno=7)
          24	BINARY_MULTIPLY(arg=None, lineno=7)
          26	LOAD_FAST(arg=0, lineno=7)
          28	LOAD_CONST(arg=3, lineno=7)
          30	BINARY_SUBSCR(arg=None, lineno=7)
          32	LOAD_FAST(arg=1, lineno=7)
          34	LOAD_CONST(arg=2, lineno=7)
          36	BINARY_SUBSCR(arg=None, lineno=7)
          38	BINARY_MULTIPLY(arg=None, lineno=7)
          40	BINARY_SUBTRACT(arg=None, lineno=7)
          42	LOAD_FAST(arg=2, lineno=7)
          44	LO

DEBUG:numba.core.byteflow:dispatch pc=74, inst=BINARY_MULTIPLY(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$60binary_multiply.27', '$66binary_subscr.30', '$72binary_subscr.33']
DEBUG:numba.core.byteflow:dispatch pc=76, inst=BINARY_SUBTRACT(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$60binary_multiply.27', '$74binary_multiply.34']
DEBUG:numba.core.byteflow:dispatch pc=78, inst=LOAD_FAST(arg=2, lineno=8)
DEBUG:numba.core.byteflow:stack ['$76binary_subtract.35']
DEBUG:numba.core.byteflow:dispatch pc=80, inst=LOAD_CONST(arg=2, lineno=8)
DEBUG:numba.core.byteflow:stack ['$76binary_subtract.35', '$c78.36']
DEBUG:numba.core.byteflow:dispatch pc=82, inst=STORE_SUBSCR(arg=None, lineno=8)
DEBUG:numba.core.byteflow:stack ['$76binary_subtract.35', '$c78.36', '$const80.37']
DEBUG:numba.core.byteflow:dispatch pc=84, inst=LOAD_FAST(arg=0, lineno=9)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=86, inst=LOAD_CONST(arg=4, lineno=9)
DEBUG:numba.core.bytefl

DEBUG:numba.core.ssa:==== SSA block analysis pass on 0
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x2ab1e9237d10>
DEBUG:numba.core.ssa:on stmt: a = arg(0, name=a)
DEBUG:numba.core.ssa:on stmt: b = arg(1, name=b)
DEBUG:numba.core.ssa:on stmt: $2load_global.0 = global(np: <module 'numpy' from '/home/zhangi/miniconda3/envs/perses-sims-oct2020/lib/python3.7/site-packages/numpy/__init__.py'>)
DEBUG:numba.core.ssa:on stmt: $4load_method.1 = getattr(value=$2load_global.0, attr=zeros)
DEBUG:numba.core.ssa:on stmt: $const6.2 = const(int, 3)
DEBUG:numba.core.ssa:on stmt: $8call_method.3 = call $4load_method.1($const6.2, func=$4load_method.1, args=[Var($const6.2, coordinate_numba.py:6)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: c = $8call_method.3
DEBUG:numba.core.ssa:on stmt: $const14.5 = const(int, 1)
DEBUG:numba.core.ssa:on stmt: $16binary_subscr.6 = static_getitem(value=a, index=1, index_var=$const14.5)
DEBUG:numba.core.ssa:on stmt: $const20.8 = const(

DEBUG:numba.core.byteflow:pending: deque([State(pc_initial=0 nstack_initial=0)])
DEBUG:numba.core.byteflow:stack: []
DEBUG:numba.core.byteflow:dispatch pc=0, inst=NOP(arg=None, lineno=14)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=2, inst=LOAD_GLOBAL(arg=0, lineno=14)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=4, inst=LOAD_METHOD(arg=1, lineno=14)
DEBUG:numba.core.byteflow:stack ['$2load_global.0']
DEBUG:numba.core.byteflow:dispatch pc=6, inst=LOAD_FAST(arg=0, lineno=14)
DEBUG:numba.core.byteflow:stack ['$4load_method.1']
DEBUG:numba.core.byteflow:dispatch pc=8, inst=LOAD_FAST(arg=0, lineno=14)
DEBUG:numba.core.byteflow:stack ['$4load_method.1', '$a6.2']
DEBUG:numba.core.byteflow:dispatch pc=10, inst=CALL_METHOD(arg=2, lineno=14)
DEBUG:numba.core.byteflow:stack ['$4load_method.1', '$a6.2', '$a8.3']
DEBUG:numba.core.byteflow:dispatch pc=12, inst=STORE_FAST(arg=1, lineno=14)
DEBUG:numba.core.byteflow:stack ['$10call_method.4']
D

DEBUG:numba.core.byteflow:stack ['$6unpack_sequence.2']
DEBUG:numba.core.byteflow:dispatch pc=10, inst=LOAD_FAST(arg=1, lineno=509)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=12, inst=LOAD_ATTR(arg=0, lineno=509)
DEBUG:numba.core.byteflow:stack ['$b10.4']
DEBUG:numba.core.byteflow:dispatch pc=14, inst=UNPACK_SEQUENCE(arg=1, lineno=509)
DEBUG:numba.core.byteflow:stack ['$12load_attr.5']
DEBUG:numba.core.byteflow:dispatch pc=16, inst=STORE_FAST(arg=3, lineno=509)
DEBUG:numba.core.byteflow:stack ['$14unpack_sequence.6']
DEBUG:numba.core.byteflow:dispatch pc=18, inst=LOAD_FAST(arg=2, lineno=510)
DEBUG:numba.core.byteflow:stack []
DEBUG:numba.core.byteflow:dispatch pc=20, inst=LOAD_FAST(arg=3, lineno=510)
DEBUG:numba.core.byteflow:stack ['$m18.8']
DEBUG:numba.core.byteflow:dispatch pc=22, inst=COMPARE_OP(arg=3, lineno=510)
DEBUG:numba.core.byteflow:stack ['$m18.8', '$n20.9']
DEBUG:numba.core.byteflow:dispatch pc=24, inst=POP_JUMP_IF_FALSE(arg=34, lineno=510)
DE

...

DEBUG:numba.core.ssa:on stmt: $140load_method.13 = getattr(value=$138load_global.12, attr=dot)
DEBUG:numba.core.ssa:on stmt: $146call_method.16 = call $140load_method.13(plane1, plane2, func=$140load_method.13, args=[Var(plane1, coordinate_numba.py:126), Var(plane2, coordinate_numba.py:127)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: $148load_global.17 = global(_norm: CPUDispatcher(<function _norm at 0x2ab1e9463b00>))
DEBUG:numba.core.ssa:on stmt: $152call_function.19 = call $148load_global.17(plane1, func=$148load_global.17, args=[Var(plane1, coordinate_numba.py:126)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: $154load_global.20 = global(_norm: CPUDispatcher(<function _norm at 0x2ab1e9463b00>))
DEBUG:numba.core.ssa:on stmt: $158call_function.22 = call $154load_global.20(plane2, func=$154load_global.20, args=[Var(plane2, coordinate_numba.py:127)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: $160binary_multiply.23 = $152call_function.19 * $158call_function.22
DE

DEBUG:numba.core.ssa:on stmt: c_u = $58binary_true_divide.23
DEBUG:numba.core.ssa:on stmt: $62load_global.24 = global(_norm: CPUDispatcher(<function _norm at 0x2ab1e9463b00>))
DEBUG:numba.core.ssa:on stmt: $66call_function.26 = call $62load_global.24(a, func=$62load_global.24, args=[Var(a, coordinate_numba.py:106)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: r = $66call_function.26
DEBUG:numba.core.ssa:on stmt: $70load_global.27 = global(np: <module 'numpy' from '/home/zhangi/miniconda3/envs/perses-sims-oct2020/lib/python3.7/site-packages/numpy/__init__.py'>)
DEBUG:numba.core.ssa:on stmt: $72load_method.28 = getattr(value=$70load_global.27, attr=dot)
DEBUG:numba.core.ssa:on stmt: $78call_method.31 = call $72load_method.28(a_u, b_u, func=$72load_method.28, args=[Var(a_u, coordinate_numba.py:110), Var(b_u, coordinate_numba.py:111)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: cos_theta = $78call_method.31
DEBUG:numba.core.ssa:on stmt: $const84.33 = const(float, 1.0)
DEBUG:

DEBUG:numba.core.ssa:on stmt: $210call_method.8 = call $204load_method.5(a, plane2, func=$204load_method.5, args=[Var(a, coordinate_numba.py:106), Var(plane2, coordinate_numba.py:127)], kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: $const212.9 = const(int, 0)
DEBUG:numba.core.ssa:on stmt: $214compare_op.10 = $210call_method.8 <= $const212.9
DEBUG:numba.core.ssa:on stmt: bool216 = global(bool: <class 'bool'>)
DEBUG:numba.core.ssa:on stmt: $216pred = call bool216($214compare_op.10, func=bool216, args=(Var($214compare_op.10, coordinate_numba.py:137),), kws=(), vararg=None)
DEBUG:numba.core.ssa:on stmt: branch $216pred, 218, 224
DEBUG:numba.core.ssa:==== SSA block rewrite pass on 218
DEBUG:numba.core.ssa:Running <numba.core.ssa._FixSSAVars object at 0x2ab1e9a86090>
DEBUG:numba.core.ssa:on stmt: $220unary_negative.1 = unary(fn=<built-in function neg>, value=phi)
DEBUG:numba.core.ssa:find_def var='phi' stmt=$220unary_negative.1 = unary(fn=<built-in function neg>, value=phi)
DEBUG:numba.

DEBUG:numba.core.byteflow:defmap: {}
DEBUG:numba.core.byteflow:phismap: defaultdict(<class 'set'>, {})
DEBUG:numba.core.byteflow:changing phismap: defaultdict(<class 'set'>, {})
DEBUG:numba.core.byteflow:keep phismap: {}
DEBUG:numba.core.byteflow:new_out: defaultdict(<class 'dict'>, {})
DEBUG:numba.core.byteflow:----------------------DONE Prune PHIs-----------------------
DEBUG:numba.core.byteflow:block_infos State(pc_initial=0 nstack_initial=0):
AdaptBlockInfo(insts=((0, {}), (2, {'res': '$angle_position_12.0'}), (4, {'res': '$bond_position_24.1'}), (6, {'lhs': '$angle_position_12.0', 'rhs': '$bond_position_24.1', 'res': '$6binary_subtract.2'}), (8, {'retval': '$6binary_subtract.2', 'castval': '$8return_value.3'})), outgoing_phis={}, blockstack=(), active_try_block=None, outgoing_edgepushed={})
DEBUG:numba.core.interpreter:label 0:
    angle_position_1 = arg(0, name=angle_position_1) ['angle_position_1']
    bond_position_2 = arg(1, name=bond_position_2) ['bond_position_2']
    $6bina

DEBUG:numba.core.ssa:==== SSA block analysis pass on 0
DEBUG:numba.core.ssa:Running <numba.core.ssa._GatherDefsHandler object at 0x2ab1e9b66890>
DEBUG:numba.core.ssa:on stmt: _32call_function_12_1 = arg(0, name=_32call_function_12_1)
DEBUG:numba.core.ssa:on stmt: a_1 = arg(1, name=a_1)
DEBUG:numba.core.ssa:on stmt: $6binary_true_divide.2 = a_1 / _32call_function_12_1
DEBUG:numba.core.ssa:on stmt: $8return_value.3 = cast(value=$6binary_true_divide.2)
DEBUG:numba.core.ssa:on stmt: return $8return_value.3
DEBUG:numba.core.ssa:defs defaultdict(<class 'list'>,
            {'$6binary_true_divide.2': [<numba.core.ir.Assign object at 0x2ab1e988bcd0>],
             '$8return_value.3': [<numba.core.ir.Assign object at 0x2ab1e988be10>],
             '_32call_function_12_1': [<numba.core.ir.Assign object at 0x2ab1e988b210>],
             'a_1': [<numba.core.ir.Assign object at 0x2ab1e988b890>]})
DEBUG:numba.core.ssa:SSA violators set()
DEBUG:numba.core.byteflow:bytecode dump:
>          0	NOP(arg=

In [3]:
# Determine indices for solute region
for atom in htf.hybrid_topology.atoms:
    print(atom.index, atom)

0 ACE1-C
1 ACE1-O
2 ACE1-CH3
3 ACE1-H1
4 ACE1-H2
5 ACE1-H3
6 THR2-N
7 THR2-CA
8 THR2-C
9 THR2-O
10 THR2-CB
11 THR2-CG2
12 THR2-OG1
13 THR2-H
14 THR2-HA
15 THR2-HB
16 THR2-HG1
17 THR2-HG21
18 THR2-HG22
19 THR2-HG23
26 THR2-CB
27 THR2-HB1
28 THR2-HB2
29 THR2-HB3
20 NME3-N
21 NME3-C
22 NME3-H
23 NME3-H1
24 NME3-H2
25 NME3-H3


In [4]:
# Check forces to see if CMMForce needs to be removed
for force in range(htf.hybrid_system.getNumForces()):
    print(htf.hybrid_system.getForce(force))

<simtk.openmm.openmm.HarmonicBondForce; proxy of <Swig Object of type 'OpenMM::HarmonicBondForce *' at 0x2ab18fcf3de0> >
<simtk.openmm.openmm.HarmonicAngleForce; proxy of <Swig Object of type 'OpenMM::HarmonicAngleForce *' at 0x2ab18fcf3de0> >
<simtk.openmm.openmm.PeriodicTorsionForce; proxy of <Swig Object of type 'OpenMM::PeriodicTorsionForce *' at 0x2ab1c253bb70> >
<simtk.openmm.openmm.NonbondedForce; proxy of <Swig Object of type 'OpenMM::NonbondedForce *' at 0x2ab18fcf3de0> >


In [5]:
# Build REST factory
factory = RESTTopologyFactory(htf.hybrid_system, solute_region=list(range(6, 10)) + [0, 1, 20, 22])

INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=2.0, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=2.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=2.0), unit=nanometer)]
INFO:REST:No unknown forces.


In [6]:
# Get REST system
REST_system = factory.REST_system

In [7]:
# Create states for each replica
n_replicas = 12  # Number of temperature replicas.
T_min = 298.0 * unit.kelvin  # Minimum temperature.
T_max = 600.0 * unit.kelvin  # Maximum temperature.
temperatures = [T_min + (T_max - T_min) * (math.exp(float(i) / float(n_replicas-1)) - 1.0) / (math.e - 1.0)
                for i in range(n_replicas)]

In [8]:
# Create reference thermodynamic state
lambda_zero_alchemical_state = RESTState.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate, composable_states=[lambda_zero_alchemical_state])

In [9]:
# Create thermodynamics states
sampler_state =  SamplerState(htf.hybrid_positions, box_vectors=htf.hybrid_system.getDefaultPeriodicBoxVectors())
beta_0 = 1/(kB*T_min)
thermodynamic_state_list = []
for temperature in temperatures:
    beta_m = 1/(kB*temperature)
    compound_thermodynamic_state_copy = copy.deepcopy(compound_thermodynamic_state)
    compound_thermodynamic_state_copy.set_alchemical_parameters(beta_0, beta_m)
    thermodynamic_state_list.append(compound_thermodynamic_state_copy)

In [14]:
# Set up sampler
length = 5 # 5 ns
move = mcmc.GHMCMove(timestep=4.0*unit.femtoseconds, n_steps=250)
simulation = multistate.ReplicaExchangeSampler(mcmc_moves=move, number_of_iterations=length*1000)

# Run t-repex
reporter_file = os.path.join(out_dir, f"0_vacuum_thr_{length}ns.nc")
reporter = multistate.MultiStateReporter(reporter_file, checkpoint_interval=1)
simulation.create(thermodynamic_states=thermodynamic_state_list,
                  sampler_states=sampler_state,
                  storage=reporter)
simulation.run()

DEBUG:openmmtools.multistate.multistatereporter:Initial checkpoint file automatically chosen as /data/chodera/zhangi/perses_benchmark/neq/11/0/0_vacuum_thr_5ns_checkpoint.nc
DEBUG:mpiplus.mpiplus:Cannot find MPI environment. MPI disabled.
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.storage_exists of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2ab1e9883090>>
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._display_citations at 0x2ab1c4d660e0>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._display_citations at 0x2ab1c4d17170>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._initialize_reporter at 0x2ab1c4d17440>
DEBUG:openmmtools.multistate.multistatereporter:Serialized state thermodynamic_states/0 is  5403B | 5.276KB | 0.005MB
DEBUG:openmmtools.utils:Storing thermodynamic states took    0.034s
DEBUG:openmmtools.multistate.multistatesampler:Sto

Please cite the following:

        Friedrichs MS, Eastman P, Vaidyanathan V, Houston M, LeGrand S, Beberg AL, Ensign DL, Bruns CM, and Pande VS. Accelerating molecular dynamic simulations on graphics processing unit. J. Comput. Chem. 30:864, 2009. DOI: 10.1002/jcc.21209
        Eastman P and Pande VS. OpenMM: A hardware-independent framework for molecular simulations. Comput. Sci. Eng. 12:34, 2010. DOI: 10.1109/MCSE.2010.27
        Eastman P and Pande VS. Efficient nonbonded interactions for molecular dynamics on a graphics processing unit. J. Comput. Chem. 31:1268, 2010. DOI: 10.1002/jcc.21413
        Eastman P and Pande VS. Constant constraint matrix approximation: A robust, parallelizable constraint method for molecular simulations. J. Chem. Theor. Comput. 6:434, 2010. DOI: 10.1021/ct900463w
        Chodera JD and Shirts MR. Replica exchange and expanded ensemble simulations as Gibbs multistate: Simple improvements for enhanced mixing. J. Chem. Phys., 135:194110, 2011. DOI:10.1063/

DEBUG:openmmtools.utils:Computing energy matrix took    8.075s
DEBUG:mpiplus.mpiplus:Single node: executing <bound method MultiStateReporter.write_energies of <openmmtools.multistate.multistatereporter.MultiStateReporter object at 0x2ab1e9883090>>
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:openmmtools.multistate.multistatesampler:Iteration 1/5000
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2ab1c4d56f80>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.029s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 41472/41472 attempted swaps (100.0%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _pr

DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took    0.938s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.048s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2ab1c4d17560>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2ab1c4d17830>
DEBUG:openmmtools.utils:Storing sampler states took    0.002s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.016s
DEBUG:openmmtools.multistate.multistatesampler:Not enough iterations for online analysis (self.online_analysis_minimum_iterations = 200)
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._online_analysis at 0x2ab1

DEBUG:openmmtools.multistate.multistatesampler:Iteration 8/5000
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2ab1c4d56f80>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.001s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 25414/41472 attempted swaps (61.3%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took    0.954s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.046s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteratio

DEBUG:openmmtools.multistate.multistatesampler:        -0.0    24.2    24.7    24.7    24.7    24.7    24.7    24.7    24.7    24.7    24.7    24.7
DEBUG:openmmtools.utils:Computing online free energy estimate took    0.005s
DEBUG:openmmtools.multistate.multistatesampler:Iteration took 1.028s.
DEBUG:openmmtools.multistate.multistatesampler:Estimated completion in 2:01:43.678044, at Tue Dec 15 16:57:48 2020 (consuming total wall clock time 2:01:59.781563).
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:openmmtools.multistate.multistatesampler:Iteration 12/5000
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2ab1c4d56f80>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas t

DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2ab1c4d17830>
DEBUG:openmmtools.utils:Storing sampler states took    0.002s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.015s
DEBUG:openmmtools.multistate.multistatesampler:Not enough iterations for online analysis (self.online_analysis_minimum_iterations = 200)
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._online_analysis at 0x2ab1c4d1a3b0>
DEBUG:openmmtools.multistate.multistatesampler:*** ONLINE analysis free energies:
DEBUG:openmmtools.multistate.multistatesampler:        -0.0    27.4    27.9    28.0    28.0    28.0    28.0    28.0    28.0    28.0    28.0    28.0
DEBUG:openmmtools.utils:Computing online free energy estimate took    0.005s
DEBUG:openmmtools.multistate.multistatesampler:Iteration took 1.040s.
DEBUG:openmmtools.multistate.multistatesampler:Estimated completion in 1:52:20.396019, at Tue Dec 15 16:48:29 2020 (con

DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took    0.883s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.046s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration at 0x2ab1c4d17560>
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iteration_items at 0x2ab1c4d17830>
DEBUG:openmmtools.utils:Storing sampler states took    0.002s
DEBUG:openmmtools.utils:Writing iteration information to storage took    0.015s
DEBUG:openmmtools.multistate.multistatesampler:Not enough iterations for online analysis (self.online_analysis_minimum_iterations = 200)
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._online_analysis at 0x2ab1

DEBUG:openmmtools.multistate.multistatesampler:Iteration 23/5000
DEBUG:openmmtools.multistate.multistatesampler:********************************************************************************
DEBUG:mpiplus.mpiplus:Single node: executing <function ReplicaExchangeSampler._mix_replicas at 0x2ab1c4d56f80>
DEBUG:openmmtools.multistate.replicaexchange:Mixing replicas...
DEBUG:openmmtools.utils:Mixing of replicas took    0.001s
DEBUG:openmmtools.multistate.replicaexchange:Accepted 23556/41472 attempted swaps (56.8%)
DEBUG:openmmtools.multistate.multistatesampler:Propagating all replicas...
DEBUG:mpiplus.mpiplus:Running _propagate_replica serially.
DEBUG:mpiplus.mpiplus:Running _get_replica_move_statistics serially.
DEBUG:openmmtools.utils:Propagating all replicas took    0.938s
DEBUG:mpiplus.mpiplus:Running _compute_replica_energies serially.
DEBUG:openmmtools.utils:Computing energy matrix took    0.046s
DEBUG:mpiplus.mpiplus:Single node: executing <function MultiStateSampler._report_iterati

KeyboardInterrupt: 